# Installs

In [ ]:
!pip install numpy
!pip install --upgrade gspread google-auth
!pip install gspread
!pip install --upgrade pandas
!pip install icalendar
!pip install pytz
!pip install py

# Code

In [ ]:
from icalendar import Calendar, Event, vText
from datetime import datetime
import pytz
import pandas as pd
from pathlib import Path
import shutil
from google.colab import auth
import gspread
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

def parse_schedule(schedule):
    df = pd.DataFrame(schedule, columns=['Date', 'Name', 'Year', 'Month', 'Day', 'Start', 'End'])

    df['Name'] = df['Name'].str.lower()
    df[['Year', 'Month', 'Day', 'Start', 'End']] = df[['Year', 'Month', 'Day', 'Start', 'End']].apply(pd.to_numeric, errors='coerce')
    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']], errors='coerce')
    return df

def create_employee_calendars(df):
    employees = df['Name'].unique()
    calendars = {}
    for e in employees:
        if e:
            calendars[e] = Calendar()
            calendars[e].add('prodid', '-//Sussex Insurance Shift Schedule//EN')
            calendars[e].add('version', '2.0')
            calendars[e].add('X-WR-CALNAME', f"{e.title()}'s Shift Schedule")
            calendars[e].add('X-WR-TIMEZONE', 'Canada/Pacific')
    return calendars

def create_icalendar(calendars, df):
    timezone = pytz.timezone("Canada/Pacific")
    for _, row in df.iterrows():
        if pd.notna(row['Start']) and pd.notna(row['End']) and row['Name']:
            event = Event()
            event.add('summary', 'Shift')
            event.add('description', 'Shift')
            event['location'] = vText('Address')

            start_time = timezone.localize(datetime(int(row['Year']), int(row['Month']), int(row['Day']), int(row['Start']), 0))
            end_time = timezone.localize(datetime(int(row['Year']), int(row['Month']), int(row['Day']), int(row['End']), 0))

            event.add('dtstart', start_time)
            event.add('dtend', end_time)

            if row['Name'] in calendars:
                calendars[row['Name']].add_component(event)

def save_calendars(calendars, df):
    output_dir = Path('calendars')
    output_dir.mkdir(exist_ok=True)

    for employee_name, calendar in calendars.items():
        if employee_name != 'stat':
            # Get the unique year and month for the shifts of each employee
            employee_df = df[df['Name'] == employee_name]
            for _, row in employee_df.iterrows():
                sanitized_name = employee_name.replace(' ', '_')
                year = int(row['Year'])
                month = int(row['Month'])

                file_path = output_dir / f"{year}_{month}_{sanitized_name}_shifts.ics"
                with open(file_path, 'wb') as f:
                    f.write(calendar.to_ical())
    print(f"Calendars saved to {output_dir}")

def main():
    try:
        cal = gc.open('Schedule')
        sheet = cal.sheet1
        schedule = sheet.get_all_values()

        df = parse_schedule(schedule)

        calendars = create_employee_calendars(df)
        create_icalendar(calendars, df)
        save_calendars(calendars, df)

        shutil.make_archive('calendars', 'zip', 'calendars')
        print("Calendars successfully zipped.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


Calendars saved to calendars
Calendars successfully zipped.
